# Dependências

In [2]:
!pip install pandas

In [3]:
!pip install psycopg2

In [4]:
!pip install fastparquet

# Bibliotecas

In [5]:
import pandas as pd
import numpy as np
import psycopg2 as postgres

# Funções

In [6]:
def generateSingleLabel(multi_label):
    
    #Análise do primeiro campo de categorias casos especiais
    if ("Beer, Wine & Spirits, " in multi_label):
        single_label = multi_label.replace("Beer, Wine & Spirits, ", '')
        single_label = single_label.split(', ')
        single_label.append("Beer, Wine & Spirits")
    elif (", Beer, Wine & Spirits, " in multi_label):
        single_label = multi_label.replace(", Beer, Wine & Spirits, ", '')
        single_label = single_label.split(', ')
        single_label.append("Beer, Wine & Spirits")
    elif (", Beer, Wine & Spirits" in multi_label):
        single_label = multi_label.replace(", Beer, Wine & Spirits", '')
        single_label = single_label.split(', ')
        single_label.append("Beer, Wine & Spirits")
                                           
    elif ("Wills, Trusts, & Probates, " in multi_label):
        single_label = multi_label.replace("Wills, Trusts, & Probates, ", '')
        single_label = single_label.split(', ')
        single_label.append("Wills, Trusts, & Probates")
    elif (", Wills, Trusts, & Probates, " in multi_label):
        single_label = multi_label.replace(", Wills, Trusts, & Probates, ", '')
        single_label = single_label.split(', ')
        single_label.append("Wills, Trusts, & Probates")
    elif (", Wills, Trusts, & Probates" in multi_label):
        single_label = multi_label.replace(", Wills, Trusts, & Probates", '')
        single_label = single_label.split(', ')
        single_label.append("Wills, Trusts, & Probates")
                                           
    elif ("Books, Mags, Music & Video, " in multi_label):
        single_label = multi_label.replace("Books, Mags, Music & Video, ", '')
        single_label = single_label.split(', ')
        single_label.append("Books, Mags, Music & Video")
    elif (", Books, Mags, Music & Video, " in multi_label):
        single_label = multi_label.replace(", Books, Mags, Music & Video, ", '')
        single_label = single_label.split(', ')
        single_label.append("Books, Mags, Music & Video")
    elif (", Books, Mags, Music & Video" in multi_label):
        single_label = multi_label.replace(", Books, Mags, Music & Video", '')
        single_label = single_label.split(', ')
        single_label.append("Books, Mags, Music & Video")
                                           
    elif ("Used, Vintage & Consignment, " in multi_label):
        single_label = multi_label.replace("Used, Vintage & Consignment, ", '')
        single_label = single_label.split(', ')
        single_label.append("Used, Vintage & Consignment")
    elif (", Used, Vintage & Consignment, " in multi_label):
        single_label = multi_label.replace(", Used, Vintage & Consignment, ", '')
        single_label = single_label.split(', ')
        single_label.append("Used, Vintage & Consignment")
    elif (", Used, Vintage & Consignment" in multi_label):
        single_label = multi_label.replace(", Used, Vintage & Consignment", '')
        single_label = single_label.split(', ')
        single_label.append("Used, Vintage & Consignment")
        
    #Casos em que a string são as palavras
    else:
        
        if (multi_label == "Beer, Wine & Spirits" or
           multi_label == "Wills, Trusts, & Probates" or
           multi_label == "Books, Mags, Music & Video" or
           multi_label == "Used, Vintage & Consignment"):
            
            single_label = multi_label
        else:
            single_label = multi_label.split(', ')  
        
    return single_label

In [7]:
def connect():
    conn = None
    try:
        conn = postgres.connect(
            host="localhost",
            database="austin_test",
            user="postgres",
            password="root")
    except postgres.Error as e:
        print(e)
    return conn


def closeConnection(conn):
    sucess = False
    try:
        conn.close()
        sucess = True
    except postgres.Error as e:
        print(e)
    
    return sucess


def executeQuery(conn, sql):
    record = None
    try:
        #print(sql)
        cur = conn.cursor()
        cur.execute(sql)
        record = cur.fetchall()
        cur.close()
    except postgres.Error as e:
        print(e)
        cur.execute("ROLLBACK")
        cur.close()
    return record


def executeInsert(conn, sql):
    sucess = False
            
    try:
        #print(sql)
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
        cur.close()
        sucess = True
    except postgres.Error as e:
        print(e)
        cur.execute("ROLLBACK")
        cur.close()

    return sucess

In [8]:
def constructBin(conn, bin_number, h):

    
    bin_range = [(bin_number*h),  ((bin_number + 1)*h)]

    sql_query = """
        Select id, ST_Difference(ST_Buffer(geom::geography, """+str(bin_range[1])+""")::geometry,ST_Buffer(geom::geography, """+str(bin_range[0])+""")::geometry) from poi;
    """

    result = executeQuery(conn, sql_query)
    for id, data in enumerate(result):

        sql_insert = """
            Insert into bin (fk_poi_id_center, number, geom) values (\'"""+str(data[0])+"""\', """+str(bin_number)+""", \'"""+str(data[1])+"""\')
        """

        response = executeInsert(conn, sql_insert)
        #print(id, "poi:", data[0], "construct status:", response)
        #break

In [9]:
def intersectionBin(conn, bin_number):
    sql_query = """
        Select id from poi;
    """

    result_01 = executeQuery(conn, sql_query)
    for b_id in result_01:

        sql_query = """
            SELECT poi.id,
            ST_DistanceSphere(poi.geom, (Select geom from poi where id = \'"""+str(b_id[0])+"""\'))
            FROM bin, poi
            WHERE ST_Contains(bin.geom, poi.geom) AND
            bin.number = """+str(bin_number)+""" AND
            bin.fk_poi_id_center = \'"""+str(b_id[0])+"""\';
        """

        #print(sql_query)

        result_02 = executeQuery(conn, sql_query)

        for data in result_02:

            if(data[0] != b_id[0]):
                sql_insert = """ 
                    Insert into has_context_poi (fk_poi_id_center, fk_poi_id_context, fk_bin_number, distance_m) 
                    values (\'"""+str(b_id[0])+"""\', \'"""+str(data[0])+"""\', \'"""+str(bin_number)+"""\', """+str(data[1])+""");
                """
                executeInsert(conn, sql_insert)

In [1]:
def insertCategory(conn):
    sql_query = "Select id, categories from poi order by id asc"
    result = executeQuery(conn, sql_query)

    for data in result:
        categories_split = generateSingleLabel(data[1])

        for category in categories_split:

            category = category.replace('\'', '\'\'')

            try:
                sql_query = """Select id from category where name = \'"""+str(category)+"""\';"""
                #print(sql_query)
                category_result = executeQuery(conn, sql_query)

                #print(data)
                sql_insert = """Insert into has_category(fk_category_id, fk_poi_id) values (\'""" +str(category_result[0][0])+ """\', """ + str(data[0]) + """);"""

                executeInsert(conn, sql_insert)
            except Exception as e:
                print(sql_query)
                print(category, category_result)

# Associando Categorias

In [64]:
conn = connect()
insertCategory(conn)
closeConnection(conn)

Select id from category where name = 'Ethnic Food';
Ethnic Food []
Select id from category where name = 'Dry Cleaning & Laundry';
Dry Cleaning & Laundry []
Select id from category where name = 'Dry Cleaning & Laundry';
Dry Cleaning & Laundry []
Select id from category where name = 'Ethnic Food';
Ethnic Food []
Select id from category where name = 'Ethnic Food';
Ethnic Food []
Select id from category where name = 'Ethnic Food';
Ethnic Food []
Select id from category where name = 'Rolfing';
Rolfing []
Select id from category where name = 'Dry Cleaning & Laundry';
Dry Cleaning & Laundry []
Select id from category where name = 'Dry Cleaning & Laundry';
Dry Cleaning & Laundry []
Select id from category where name = 'Dry Cleaning & Laundry';
Dry Cleaning & Laundry []
Select id from category where name = 'Ethnic Food';
Ethnic Food []
Select id from category where name = 'Dry Cleaning & Laundry';
Dry Cleaning & Laundry []
Select id from category where name = 'Dry Cleaning & Laundry';
Dry Clean

True

# Criando bins

In [10]:
conn = connect()
for n in range(17, 27):
    #print('Construction shape of bin:', n)
    #constructBin(conn, n, 100)
        
    print('Construction intersection bin:', n)
    intersectionBin(conn, n)
    #break

print('Close connection:', closeConnection(conn))
print('Process finished.')


Construction intersection bin: 17
Construction intersection bin: 18
Construction intersection bin: 19
Construction intersection bin: 20
Construction intersection bin: 21
Construction intersection bin: 22
Construction intersection bin: 23
Construction intersection bin: 24
Construction intersection bin: 25
Construction intersection bin: 26
Close connection: True
Process finished.


# Lendo Dados

In [2]:
#distance_file_name = 'C:/Users/tield/Documents/Place2vec Experiments/Cities/austin-ml-updiated-distance-matrix-reduced.parquet'
distance_file_name = 'C:/Users/tield/Documents/Place2vec Experiments/Cities/austin-ml-updated-distance-matrix-slim.parquet'
pois_file_name = 'C:/Users/tield/Documents/Place2vec Experiments/Cities/austin-ml-updated.csv'

In [3]:
pois = pd.read_csv(pois_file_name)
print(pois.shape)
pois.head()

(22399, 7)


,business_id,city,state,latitude,longitude,categories,checkin_count
0,N3_Gs3DnX4k9SgpwJxdEfw,Austin,TX,30.346169,-97.711458,"Shopping, Jewelry Repair, Appraisal Services, ...",14
1,tXvdYGvlEceDljN8gt2_3Q,Austin,TX,30.172706,-97.799920,"Barbers, Beauty & Spas",1
2,nTIhpR7MhsALPwg_Hh14EA,Austin,TX,30.326377,-97.704543,"Hotels, Hotels & Travel, Event Planning & Serv...",475
3,8XyEpVdAO0o6iVkVxkWosQ,Austin,TX,30.246465,-97.778738,"Home Services, Real Estate, Property Management",0
4,NVfOn7TdnHbaGH97CVB_Qg,Austin,TX,30.244902,-97.857409,"Chiropractors, Health & Medical",33


In [4]:
distance_matrix = pd.read_parquet(distance_file_name)
print(distance_matrix.shape)
distance_matrix.head()

(22399, 22399)


,0,1,2,3,4,5,6,7,8,9,...,22389,22390,22391,22392,22393,22394,22395,22396,22397,22398
0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,21076.56,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2298.67,19388.36,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12831.06,8450.43,11388.95,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17976.42,9744.97,17248.78,7559.03,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Parciais

In [ ]:
#Opcional
for n in range(0, 24000, 100):
    data = pd.read_csv(distance_file_name, skiprows=range(0, n), nrows=100)
    print(n, data.shape)
    #break
#data.head()

# Inserindo Informações na Base de Dados

In [6]:
#Construindo a matriz de distâncias
#i = 0
max_distance = 3000
for idr, row in distance_matrix.iterrows():
    print('Execunting row:', idr)
    for idc, val in enumerate(row):
        
        if (idc < idr):
            if (val <= max_distance):

                poi_01 = pois.iloc[idr]['business_id']
                poi_02 = pois.iloc[idc]['business_id']
                sql = 'Insert into has_distance(fk_POI_business_id_01, fk_POI_business_id_02, distance_m) values (\'' + str(poi_01) + '\', \'' + str(poi_02) + '\', ' + str(val) + ');'
                #print(sql)
                
                #Inserindo na base
                try:
                    cur = conn.cursor()
                    cur.execute(sql)
                    conn.commit()
                    cur.close()
                except postgres.Error as e:
                    print(e)
                    cur.execute("ROLLBACK")
                    cur.close()
        else:
            break

print('process finished.')

Execunting row: 0
Execunting row: 1
Execunting row: 2
Execunting row: 3
Execunting row: 4
Execunting row: 5
Execunting row: 6
Execunting row: 7
Execunting row: 8
Execunting row: 9
Execunting row: 10
Execunting row: 11
Execunting row: 12
Execunting row: 13
Execunting row: 14
Execunting row: 15
Execunting row: 16
Execunting row: 17
Execunting row: 18
Execunting row: 19
Execunting row: 20
Execunting row: 21
Execunting row: 22
Execunting row: 23
Execunting row: 24
Execunting row: 25
Execunting row: 26
Execunting row: 27
Execunting row: 28
Execunting row: 29
Execunting row: 30
Execunting row: 31
Execunting row: 32
Execunting row: 33
Execunting row: 34
Execunting row: 35
Execunting row: 36
Execunting row: 37
Execunting row: 38
Execunting row: 39
Execunting row: 40
Execunting row: 41
Execunting row: 42
Execunting row: 43
Execunting row: 44
Execunting row: 45
Execunting row: 46
Execunting row: 47
Execunting row: 48
Execunting row: 49
Execunting row: 50
Execunting row: 51
Execunting row: 52
Exe